In [1]:
# !pip install tensorflow==2.16.1
# pip install numpy==1.26.4

import json
import os
import sys
import asyncio
import argparse
from collections import defaultdict
import time

os.environ["JAX_PLATFORMS"] = "cpu"
from etils import epath
import json
import base64

import torch
import numpy as np
import jax.numpy as jnp
import jax
import orbax
import orbax.checkpoint as ocp
from etils import epath
from jax.sharding import PartitionSpec as PS
from flax.traverse_util import flatten_dict, unflatten_dict


METADATA_FILE = '_METADATA'
_CHECKPOINT_FILE = 'checkpoint'


read_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm3.5-7b-chat-v5/checkpoints/448800/state'
save_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_1112/checkpoints'

read_dir = epath.Path(read_dir) 
save_dir = epath.Path(save_dir)

metadata_path = read_dir / METADATA_FILE
back_metadata_path = read_dir / f'{METADATA_FILE}.back'
try:
    metadata_path.rename(back_metadata_path)
except:
    pass
metadata_path.unlink(missing_ok=True) # delete
structure_path = read_dir / _CHECKPOINT_FILE
msgpack = ocp.aggregate_handlers.MsgpackHandler(0)
structure = msgpack.deserialize(structure_path)
# backup original checkpoint fil
back_structure_path = read_dir / 'checkpoint_back'
back_structure = structure.copy()
if not back_structure_path.exists():
    asyncio.run(msgpack.serialize(back_structure_path, item=back_structure))
print(f'Old structure file keys: {structure.keys()}')
remove_keys = ['opt_state', 'step'] # select the weight name you don't want to load, all weight name: opt_state, step, params
_ = [structure.pop(key) for key in remove_keys if key in structure]
print(f'New structure file keys: {structure.keys()}')
asyncio.run(msgpack.serialize(structure_path, item=structure))  # rewrite struct file

# load model based struct, note: axes must same as training
mesh_axes = ['data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'tensor', 'autoregressive']
devices = np.asarray(jax.devices()).reshape([1] * len(mesh_axes))
mesh = jax.sharding.Mesh(devices, mesh_axes)
sharding = jax.sharding.NamedSharding(mesh, PS()) # Sharding is None because we use cpu to load weights
weight_dtype = jnp.bfloat16 # set restore weights dtype
restore_args = {}
for k, v in flatten_dict(structure).items():
    restore_args[k] =  ocp.ArrayRestoreArgs(restore_type=jax.Array, dtype=weight_dtype, sharding=sharding)
restore_args = unflatten_dict(restore_args)
ckptr = ocp.Checkpointer(ocp.PyTreeCheckpointHandler())
w = ckptr.restore(read_dir, args=ocp.args.PyTreeRestore(restore_args=restore_args))
structure_path = read_dir / _CHECKPOINT_FILE
# rewrite struct file, otherwise occur error when continue training
asyncio.run(msgpack.serialize(structure_path, item=back_structure))
while 'params' in w:
    w = w['params']
xm3p5_w = {'.'.join(k): np.array(v) for k, v in flatten_dict(w).items()}

try:
    back_metadata_path.rename(metadata_path)
except:
    pass

2024-11-12 09:53:10.908390: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-12 09:53:10.923540: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-12 09:53:10.923568: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-12 09:53:13.319411: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Old structure file keys: dict_keys(['opt_state', 'params', 'step'])
New structure file keys: dict_keys(['params'])


I0000 00:00:1731405196.676033   16602 gcs_resource.cc:109] Using default AdmissionQueue with limit 32
I0000 00:00:1731405196.680023   17199 google_auth_provider.cc:180] Running on GCE, using service account 887571727717-compute@developer.gserviceaccount.com


In [7]:
## 基于dense模型保存和moe相同名字的参数
def convert_to_jnp(params, remove_keys=[]):
    convert_params = {}
    for k, v in params.items():
        r = 0
        for remove_key in remove_keys:
            if remove_key in k: 
                r = 1
                break
        if r: continue
        k = tuple(k.split('.'))
        convert_params[k] = v
        # convert_params[k] = jnp.array(v).astype(jnp.bfloat16)
    for k, v in convert_params.items():
        print(k, v.shape, v.dtype)
    return convert_params


def save_params(step, save_dir, params):
    item = {
            'state': orbax.checkpoint.AsyncCheckpointer(orbax.checkpoint.PyTreeCheckpointHandler(use_ocdbt=False)),
                    }
    # new_params = {tuple(k.split('.')): v for k,v in params.items()}
    unflatten_params = unflatten_dict(params)
    for k, v in params.items():
        print(k, v.shape, v.dtype)
    mngr = orbax.checkpoint.CheckpointManager(save_dir, item)
    if 'params' not in unflatten_params: unflatten_params = {'params': unflatten_params}
    mngr.save(step, items={'state': {'params': unflatten_params}})

convert_params = convert_to_jnp(xm3p5_w)
save_params(0, save_dir, convert_params)

('decoder', 'decoder_norm', 'scale') (4096,) bfloat16
('decoder', 'layers', 'mlp_0', 'mgate', 'kernel') (4096, 12, 44) bfloat16
('decoder', 'layers', 'mlp_0', 'wi_0', 'kernel') (4096, 12, 5632) bfloat16
('decoder', 'layers', 'mlp_0', 'wi_1', 'kernel') (4096, 12, 5632) bfloat16
('decoder', 'layers', 'mlp_0', 'wo', 'kernel') (5632, 12, 4096) bfloat16
('decoder', 'layers', 'mlp_1', 'mgate', 'kernel') (4096, 12, 44) bfloat16
('decoder', 'layers', 'mlp_1', 'wi_0', 'kernel') (4096, 12, 5632) bfloat16
('decoder', 'layers', 'mlp_1', 'wi_1', 'kernel') (4096, 12, 5632) bfloat16
('decoder', 'layers', 'mlp_1', 'wo', 'kernel') (5632, 12, 4096) bfloat16
('decoder', 'layers', 'mlp_2', 'mgate', 'kernel') (4096, 12, 44) bfloat16
('decoder', 'layers', 'mlp_2', 'wi_0', 'kernel') (4096, 12, 5632) bfloat16
('decoder', 'layers', 'mlp_2', 'wi_1', 'kernel') (4096, 12, 5632) bfloat16
('decoder', 'layers', 'mlp_2', 'wo', 'kernel') (5632, 12, 4096) bfloat16
('decoder', 'layers', 'mlp_3', 'mgate', 'kernel') (4096

In [50]:
## moe部分的参数保存，保存后在bucket后台人工进行转移
start_time = time.time()
unshared_experts = 44

scale = 1
mlp_dim = 5632 // scale
model_dim = 4096 // scale
copy_dim = 128 // scale
copy_expert_dim = mlp_dim // unshared_experts
# fp16_dtype = np.dtype('float16')
# 4个子层
total_moe_params = [{} for i in range(4)]
for k, v in xm3p5_w.items():
    v = jnp.array(v).astype(jnp.bfloat16)
    print(f'k: {k} take: {time.time() - start_time:.3f}s')
    if 'decoder.layers.mlp_' in k:
        mlp_inx = k.find('mlp_')
        l = k[mlp_inx+4: mlp_inx+5]
        # if int(l) in [2, 3]:
        if int(l) in [0, 1]:
            
            continue
        if int(l) % 2 != 0: continue
        moe_params = total_moe_params[int(l)]
        unshared_mlp = k.replace('decoder.layers.mlp_', 'decoder.layers.unshared_mlp_')
        if 'mgate' in unshared_mlp:
            unshared_mlp1 = unshared_mlp.replace('.mgate', '.router_gate')
            unshared_mlp2 = unshared_mlp.replace('.kernel', '')
            moe_params[unshared_mlp1] = v
            moe_params[unshared_mlp2] = v
            continue
        unshared_mlp = unshared_mlp.replace('.kernel', '')
        v = v.transpose(1, 0, 2)
        # unshared: 44 * 12 * model_dim * mlp_dim,  mlp: model_dim * 12 * mlp_dim
        unshared_mlp_w = []
        if '.wo.' in k:
            copy_w = v.reshape(12, 44, 1, copy_dim, model_dim).repeat(44, 2).reshape(12, 44, -1, model_dim).transpose(1, 0, 2, 3)
        else:
            copy_w = v.reshape(12, model_dim, 44, 1, copy_dim).repeat(44, 3).reshape(12, model_dim, 44, -1).transpose(2, 0, 1, 3)
        v = v[None] # extend dim
        init_w = (copy_w + v) / 2
        print(f'copy_w: {copy_w.shape} copy_w: {copy_w.dtype} init_w: {init_w.dtype} unshared_mlp: {unshared_mlp}')
        moe_params[unshared_mlp] = init_w


k: decoder.decoder_norm.scale take: 0.001s
k: decoder.layers.mlp_0.mgate.kernel take: 0.004s
k: decoder.layers.mlp_0.wi_0.kernel take: 0.057s
k: decoder.layers.mlp_0.wi_1.kernel take: 0.109s
k: decoder.layers.mlp_0.wo.kernel take: 0.161s
k: decoder.layers.mlp_1.mgate.kernel take: 0.162s
k: decoder.layers.mlp_1.wi_0.kernel take: 0.215s
k: decoder.layers.mlp_1.wi_1.kernel take: 0.266s
k: decoder.layers.mlp_1.wo.kernel take: 0.318s
k: decoder.layers.mlp_2.mgate.kernel take: 0.319s
k: decoder.layers.mlp_2.wi_0.kernel take: 0.369s
copy_w: (44, 12, 4096, 5632) copy_w: bfloat16 init_w: bfloat16 unshared_mlp: decoder.layers.unshared_mlp_2.wi_0
k: decoder.layers.mlp_2.wi_1.kernel take: 21.987s
copy_w: (44, 12, 4096, 5632) copy_w: bfloat16 init_w: bfloat16 unshared_mlp: decoder.layers.unshared_mlp_2.wi_1
k: decoder.layers.mlp_2.wo.kernel take: 35.737s
copy_w: (44, 12, 5632, 4096) copy_w: bfloat16 init_w: bfloat16 unshared_mlp: decoder.layers.unshared_mlp_2.wo
k: decoder.layers.mlp_3.mgate.kernel

In [51]:
for i, params in enumerate(total_moe_params):
    if not params: continue
    print(params.keys(), '\n')
    save_moe_params = {}
    for k, v in params.items():
        newk = tuple(k.split('.'))
        save_moe_params[newk] = v
    print(f'Save step: {i+1}')
    save_params(i + 1, save_dir, save_moe_params)

dict_keys(['decoder.layers.unshared_mlp_2.router_gate.kernel', 'decoder.layers.unshared_mlp_2.mgate', 'decoder.layers.unshared_mlp_2.wi_0', 'decoder.layers.unshared_mlp_2.wi_1', 'decoder.layers.unshared_mlp_2.wo']) 

Save step: 3
('decoder', 'layers', 'unshared_mlp_2', 'router_gate', 'kernel') (4096, 12, 44) bfloat16
('decoder', 'layers', 'unshared_mlp_2', 'mgate') (4096, 12, 44) bfloat16
('decoder', 'layers', 'unshared_mlp_2', 'wi_0') (44, 12, 4096, 5632) bfloat16
('decoder', 'layers', 'unshared_mlp_2', 'wi_1') (44, 12, 4096, 5632) bfloat16
('decoder', 'layers', 'unshared_mlp_2', 'wo') (44, 12, 5632, 4096) bfloat16


In [37]:
# _METADATA, checkpoint三个文件转移到实际保存的checkpoint文件夹中进行替换
start_time = time.time()
unshared_experts = 44
mlp_dim = 5632 // scale
copy_expert_dim = mlp_dim // unshared_experts
moe_params = {}
for k, v in xm3p5_w.items():
    v = jnp.array([100]).astype(jnp.bfloat16)
    print(f'k: {k} take: {time.time() - start_time:.3f}s')
    moe_params[k] = v
    if 'decoder.layers.mlp_' in k:
        mlp_inx = k.find('mlp_')
        l = k[mlp_inx+4: mlp_inx+5]
        if int(l) % 2 != 0: continue
        # mgate需要.kernel, moe不需要，因为moe的参数采用的是self.params创建的，而mgate是采用flax的DenseGeneral
        unshared_mlp = k.replace('decoder.layers.mlp_', 'decoder.layers.unshared_mlp_')
        if 'mgate' in unshared_mlp:
            unshared_mlp1 = unshared_mlp.replace('.mgate', '.router_gate')
            unshared_mlp2 = unshared_mlp.replace('.kernel', '')
            moe_params[unshared_mlp1] = v
            moe_params[unshared_mlp2] = v
        else:
            unshared_mlp = unshared_mlp.replace('.kernel', '')
            moe_params[unshared_mlp] = v

moe_params = {tuple(k.split('.')): v for k,v in moe_params.items()}
save_params(5, save_dir, moe_params)

k: decoder.decoder_norm.scale take: 0.001s
k: decoder.layers.mlp_0.mgate.kernel take: 0.002s
k: decoder.layers.mlp_0.wi_0.kernel take: 0.002s
k: decoder.layers.mlp_0.wi_1.kernel take: 0.002s
k: decoder.layers.mlp_0.wo.kernel take: 0.002s
k: decoder.layers.mlp_1.mgate.kernel take: 0.002s
k: decoder.layers.mlp_1.wi_0.kernel take: 0.002s
k: decoder.layers.mlp_1.wi_1.kernel take: 0.003s
k: decoder.layers.mlp_1.wo.kernel take: 0.003s
k: decoder.layers.mlp_2.mgate.kernel take: 0.003s
k: decoder.layers.mlp_2.wi_0.kernel take: 0.003s
k: decoder.layers.mlp_2.wi_1.kernel take: 0.003s
k: decoder.layers.mlp_2.wo.kernel take: 0.003s
k: decoder.layers.mlp_3.mgate.kernel take: 0.003s
k: decoder.layers.mlp_3.wi_0.kernel take: 0.004s
k: decoder.layers.mlp_3.wi_1.kernel take: 0.004s
k: decoder.layers.mlp_3.wo.kernel take: 0.004s
k: decoder.layers.post_self_attention_layer_norm_0.scale take: 0.004s
k: decoder.layers.post_self_attention_layer_norm_1.scale take: 0.004s
k: decoder.layers.post_self_attention

In [38]:
import subprocess

# save_dir1 = str(save_dir).rstrip('/')
source_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_1112/checkpoints'
target_dir = 'gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_1112/checkpoints'

source_step = 5
command = f'gsutil cp {source_dir}/{source_step}/state/_METADATA {target_dir}/0/state/ '
r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

command = f'gsutil cp {source_dir}/{source_step}/state/checkpoint {target_dir}/0/state/ '
r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

# command = f'gsutil cp {source_dir}/{source_step}/state/_sharding {target_dir}/0/state/ '
# r = subprocess.run(command, stdout=subprocess.PIPE, shell=True)

Copying gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_1112/checkpoints/5/state/_METADATA [Content-Type=application/octet-stream]...
/ [1 files][ 30.3 KiB/ 30.3 KiB]                                                
Operation completed over 1 objects/30.3 KiB.                                     
Copying gs://llm_base_models_europe-west4/v5p_256/7B/xm_45x7B_moe_1112/checkpoints/5/state/checkpoint...
/ [1 files][  6.6 KiB/  6.6 KiB]                                                
Operation completed over 1 objects/6.6 KiB.                                      


In [52]:
# 基于tpu type 构建_sharding文件
'''
_sharding文件格式如下：
{
  b3B0X3N0YXRlLm11LnBhcmFtcy50b2tlbl9lbWJlZGRlci5lbWJlZGRpbmc=': {'sharding_type': 'NamedSharding',
  'shape': [1, 1, 4, 1, 1, 1, 1],
  'axis_names': ['data', 'stage', 'fsdp', 'fsdp_transpose', 'sequence', 'tensor','autoregressive'],
  'partition_spec': [['tensor', 'autoregressive'], ['fsdp', 'fsdp_transpose', 'sequence']],
   2: 4},
   ...
   }
   '''
# moe sharding
_sharding_path = 'gs://llm_base_models_us-east5/v5p_256/7B/xm_45x7B_moe_1017/xm3p5_moe_params_no_opt_v5p_64_sharding.copy'
_sharding_path = epath.Path(_sharding_path)
# 读取已有的_sharding文件
with _sharding_path.open('r') as f:
    _sharding = json.load(f)

tpu_type = 'v5p-64'
core_nums = int(tpu_type.split('-')[-1])
if 'v3' not in tpu_type:
    core_nums = core_nums // 2
print(f'core_nums: {core_nums}')
updated_sharding = {}
for k, v in _sharding.items():
    v = json.loads(v)
    v['shape'][2] = core_nums
    updated_sharding[k] = json.dumps(v)
    
updated_sharding_path = f'{save_dir}/0/state/_sharding'

updated_sharding_path = epath.Path(updated_sharding_path)
with updated_sharding_path.open('w') as f:
    json.dump(updated_sharding, f)

core_nums: 32


In [23]:
# Dense模型参数
# params.params.decoder.decoder_norm.scale
# params.params.decoder.layers.mlp_0.mgate.kernel
# params.params.decoder.layers.mlp_0.wi_0.kernel
# params.params.decoder.layers.mlp_0.wi_1.kernel
# params.params.decoder.layers.mlp_0.wo.kernel
# params.params.decoder.layers.mlp_1.mgate.kernel
# params.params.decoder.layers.mlp_1.wi_0.kernel
# params.params.decoder.layers.mlp_1.wi_1.kernel
# params.params.decoder.layers.mlp_1.wo.kernel
# params.params.decoder.layers.mlp_2.mgate.kernel
# params.params.decoder.layers.mlp_2.wi_0.kernel
# params.params.decoder.layers.mlp_2.wi_1.kernel
# params.params.decoder.layers.mlp_2.wo.kernel
# params.params.decoder.layers.mlp_3.mgate.kernel
# params.params.decoder.layers.mlp_3.wi_0.kernel
# params.params.decoder.layers.mlp_3.wi_1.kernel
# params.params.decoder.layers.mlp_3.wo.kernel
# params.params.decoder.layers.post_self_attention_layer_norm_0.scale
# params.params.decoder.layers.post_self_attention_layer_norm_1.scale
# params.params.decoder.layers.post_self_attention_layer_norm_2.scale
# params.params.decoder.layers.post_self_attention_layer_norm_3.scale
# params.params.decoder.layers.pre_self_attention_layer_norm_0.scale
# params.params.decoder.layers.pre_self_attention_layer_norm_1.scale
# params.params.decoder.layers.pre_self_attention_layer_norm_2.scale
# params.params.decoder.layers.pre_self_attention_layer_norm_3.scale
# params.params.decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.dd.kernel
# params.params.decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.dw1.kernel
# params.params.decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.qkw
# params.params.decoder.layers.self_attention_0.k_norm.scale
# params.params.decoder.layers.self_attention_0.key.kernel
# params.params.decoder.layers.self_attention_0.out.kernel
# params.params.decoder.layers.self_attention_0.q_norm.scale
# params.params.decoder.layers.self_attention_0.query.kernel
# params.params.decoder.layers.self_attention_0.value.kernel
# params.params.decoder.layers.self_attention_1.AttentionOp_0.dyn_w_proj.dd.kernel
# params.params.decoder.layers.self_attention_1.AttentionOp_0.dyn_w_proj.dw1.kernel
# params.params.decoder.layers.self_attention_1.AttentionOp_0.dyn_w_proj.qkw
# params.params.decoder.layers.self_attention_1.k_norm.scale
# params.params.decoder.layers.self_attention_1.key.kernel
# params.params.decoder.layers.self_attention_1.out.kernel
# params.params.decoder.layers.self_attention_1.q_norm.scale
# params.params.decoder.layers.self_attention_1.query.kernel
# params.params.decoder.layers.self_attention_1.value.kernel
# params.params.decoder.layers.self_attention_2.AttentionOp_0.dyn_w_proj.dd.kernel
# params.params.decoder.layers.self_attention_2.AttentionOp_0.dyn_w_proj.dw1.kernel
# params.params.decoder.layers.self_attention_2.AttentionOp_0.dyn_w_proj.qkw
# params.params.decoder.layers.self_attention_2.k_norm.scale
# params.params.decoder.layers.self_attention_2.key.kernel
# params.params.decoder.layers.self_attention_2.out.kernel
# params.params.decoder.layers.self_attention_2.q_norm.scale
# params.params.decoder.layers.self_attention_2.query.kernel
# params.params.decoder.layers.self_attention_2.value.kernel
# params.params.decoder.layers.self_attention_3.AttentionOp_0.dyn_w_proj.dd.kernel
# params.params.decoder.layers.self_attention_3.AttentionOp_0.dyn_w_proj.dw1.kernel
# params.params.decoder.layers.self_attention_3.AttentionOp_0.dyn_w_proj.qkw
# params.params.decoder.layers.self_attention_3.k_norm.scale
# params.params.decoder.layers.self_attention_3.key.kernel
# params.params.decoder.layers.self_attention_3.out.kernel
# params.params.decoder.layers.self_attention_3.q_norm.scale
# params.params.decoder.layers.self_attention_3.query.kernel
# params.params.decoder.layers.self_attention_3.value.kernel
# params.params.decoder.logits_dense.kernel
# params.params.token_embedder.embedding

# MOE模型参数
# params.params.decoder.decoder_norm.scale
# params.params.decoder.layers.mlp_0.mgate.kernel
# params.params.decoder.layers.mlp_0.wi_0.kernel
# params.params.decoder.layers.mlp_0.wi_1.kernel
# params.params.decoder.layers.mlp_0.wo.kernel
# params.params.decoder.layers.mlp_1.mgate.kernel
# params.params.decoder.layers.mlp_1.wi_0.kernel
# params.params.decoder.layers.mlp_1.wi_1.kernel
# params.params.decoder.layers.mlp_1.wo.kernel
# params.params.decoder.layers.mlp_2.mgate.kernel
# params.params.decoder.layers.mlp_2.wi_0.kernel
# params.params.decoder.layers.mlp_2.wi_1.kernel
# params.params.decoder.layers.mlp_2.wo.kernel
# params.params.decoder.layers.mlp_3.mgate.kernel
# params.params.decoder.layers.mlp_3.wi_0.kernel
# params.params.decoder.layers.mlp_3.wi_1.kernel
# params.params.decoder.layers.mlp_3.wo.kernel
# params.params.decoder.layers.post_self_attention_layer_norm_0.scale
# params.params.decoder.layers.post_self_attention_layer_norm_1.scale
# params.params.decoder.layers.post_self_attention_layer_norm_2.scale
# params.params.decoder.layers.post_self_attention_layer_norm_3.scale
# params.params.decoder.layers.pre_self_attention_layer_norm_0.scale
# params.params.decoder.layers.pre_self_attention_layer_norm_1.scale
# params.params.decoder.layers.pre_self_attention_layer_norm_2.scale
# params.params.decoder.layers.pre_self_attention_layer_norm_3.scale
# params.params.decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.dd.kernel
# params.params.decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.dw1.kernel
# params.params.decoder.layers.self_attention_0.AttentionOp_0.dyn_w_proj.qkw
# params.params.decoder.layers.self_attention_0.k_norm.scale
# params.params.decoder.layers.self_attention_0.key.kernel
# params.params.decoder.layers.self_attention_0.out.kernel
# params.params.decoder.layers.self_attention_0.q_norm.scale
# params.params.decoder.layers.self_attention_0.query.kernel
# params.params.decoder.layers.self_attention_0.value.kernel
# params.params.decoder.layers.self_attention_1.AttentionOp_0.dyn_w_proj.dd.kernel
# params.params.decoder.layers.self_attention_1.AttentionOp_0.dyn_w_proj.dw1.kernel
# params.params.decoder.layers.self_attention_1.AttentionOp_0.dyn_w_proj.qkw
# params.params.decoder.layers.self_attention_1.k_norm.scale
# params.params.decoder.layers.self_attention_1.key.kernel
# params.params.decoder.layers.self_attention_1.out.kernel
# params.params.decoder.layers.self_attention_1.q_norm.scale
# params.params.decoder.layers.self_attention_1.query.kernel
# params.params.decoder.layers.self_attention_1.value.kernel
# params.params.decoder.layers.self_attention_2.AttentionOp_0.dyn_w_proj.dd.kernel
# params.params.decoder.layers.self_attention_2.AttentionOp_0.dyn_w_proj.dw1.kernel
# params.params.decoder.layers.self_attention_2.AttentionOp_0.dyn_w_proj.qkw
# params.params.decoder.layers.self_attention_2.k_norm.scale
# params.params.decoder.layers.self_attention_2.key.kernel
# params.params.decoder.layers.self_attention_2.out.kernel
# params.params.decoder.layers.self_attention_2.q_norm.scale
# params.params.decoder.layers.self_attention_2.query.kernel
# params.params.decoder.layers.self_attention_2.value.kernel
# params.params.decoder.layers.self_attention_3.AttentionOp_0.dyn_w_proj.dd.kernel
# params.params.decoder.layers.self_attention_3.AttentionOp_0.dyn_w_proj.dw1.kernel
# params.params.decoder.layers.self_attention_3.AttentionOp_0.dyn_w_proj.qkw
# params.params.decoder.layers.self_attention_3.k_norm.scale
# params.params.decoder.layers.self_attention_3.key.kernel
# params.params.decoder.layers.self_attention_3.out.kernel
# params.params.decoder.layers.self_attention_3.q_norm.scale
# params.params.decoder.layers.self_attention_3.query.kernel
# params.params.decoder.layers.self_attention_3.value.kernel
# params.params.decoder.layers.unshared_mlp_0.mgate
# params.params.decoder.layers.unshared_mlp_0.router_gate.kernel
# params.params.decoder.layers.unshared_mlp_0.wi_0
# params.params.decoder.layers.unshared_mlp_0.wi_1
# params.params.decoder.layers.unshared_mlp_0.wo
# params.params.decoder.layers.unshared_mlp_2.mgate
# params.params.decoder.layers.unshared_mlp_2.router_gate.kernel
# params.params.decoder.layers.unshared_mlp_2.wi_0
# params.params.decoder.layers.unshared_mlp_2.wi_1
# params.params.decoder.layers.unshared_mlp_2.wo
# params.params.decoder.logits_dense.kernel
# params.params.token_embedder.embedding
